In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
data=pd.read_csv('solar_dataframe.csv')
data.head()

,Country,State,City,District,GrossPower,MainOrientation,MainOrientationTiltAngle,NetRatedPower,FeedInType,AssignedActivePowerInverter,NumberOfModules,Location,RegistrationDate,CommissioningDate,UnitOperationalStatus,UniformOrientationAndTiltAngle
0,Germany,Nordrhein-Westfalen,Münster,Münster,3.96,South,20 - 40 Grad,3.96,Full Feed-in,4.0,22.0,"Structural installations (house roof, building...",2019-02-01,2007-07-20,In operation,1.0
1,Germany,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,7.41,South,20 - 40 Grad,7.41,Partial Feed-in,8.3,38.0,"Structural installations (house roof, building...",2019-01-31,2013-01-31,In operation,1.0
2,Germany,Brandenburg,Havelland,Nauen,5.04,South,20 - 40 Grad,5.00,Partial Feed-in,5.0,16.0,"Structural installations (house roof, building...",2019-01-31,2016-02-19,In operation,1.0
3,Germany,Bavaria,Regensburg,Pentling,6.36,South-West,20 - 40 Grad,6.00,Partial Feed-in,6.0,24.0,"Structural installations (house roof, building...",2019-01-31,2016-12-16,In operation,1.0
4,Germany,Saarland,Saarlouis,Saarlouis,7.20,West,20 - 40 Grad,7.20,Partial Feed-in,7.6,30.0,"Structural installations (house roof, building...",2019-01-31,2011-12-08,In operation,0.0


In [3]:
data.columns

Index(['Country', 'State', 'City', 'District', 'GrossPower', 'MainOrientation',
       'MainOrientationTiltAngle', 'NetRatedPower', 'FeedInType',
       'AssignedActivePowerInverter', 'NumberOfModules', 'Location',
       'RegistrationDate', 'CommissioningDate', 'UnitOperationalStatus',
       'UniformOrientationAndTiltAngle'],
      dtype='object')

In [4]:
data.shape

(4642261, 16)

In [5]:
data.isnull().sum()/len(data)

Country                           0.000000
State                             0.000006
City                              0.000013
District                          0.000013
GrossPower                        0.000000
MainOrientation                   0.093385
MainOrientationTiltAngle          0.099824
NetRatedPower                     0.000000
FeedInType                        0.005320
AssignedActivePowerInverter       0.000614
NumberOfModules                   0.019877
Location                          0.000011
RegistrationDate                  0.000000
CommissioningDate                 0.012750
UnitOperationalStatus             0.000000
UniformOrientationAndTiltAngle    0.098482
dtype: float64

In [6]:
data.dropna(axis=0, inplace=True)

In [7]:
data.isnull().sum()/len(data)

Country                           0.0
State                             0.0
City                              0.0
District                          0.0
GrossPower                        0.0
MainOrientation                   0.0
MainOrientationTiltAngle          0.0
NetRatedPower                     0.0
FeedInType                        0.0
AssignedActivePowerInverter       0.0
NumberOfModules                   0.0
Location                          0.0
RegistrationDate                  0.0
CommissioningDate                 0.0
UnitOperationalStatus             0.0
UniformOrientationAndTiltAngle    0.0
dtype: float64

In [8]:
date_columns=["RegistrationDate", "CommissioningDate"]
data[date_columns]=data[date_columns].apply(pd.to_datetime)

In [9]:
data['CommissioningYear']=data['CommissioningDate'].dt.year
data['CommissioningMonth']=data['CommissioningDate'].dt.month
data['CommissioningDay']=data['CommissioningDate'].dt.day

In [10]:
data['RegistrationYear']=data['RegistrationDate'].dt.year
data['RegistrationMonth']=data['RegistrationDate'].dt.month
data['RegistrationDay']=data['RegistrationDate'].dt.day

In [11]:
data["MainOrientationTiltAngle"].value_counts()

20 - 40 Grad         2317616
40 - 60 Grad          894298
< 20 Grad             691665
Facade-integrated      38206
> 60 Grad              35043
Adjusted               10103
Name: MainOrientationTiltAngle, dtype: int64

In [12]:
# Mapping tilt categories to degree ranges
angle_ranges={
    "< 20 Grad": (0, 18),              # 0-20 degrees
    "20 - 40 Grad": (18, 36),          # 20-40 degrees
    "40 - 60 Grad": (36, 54),          # 40-60 degrees
    "> 60 Grad": (54, 90),             # 60-90 degrees
    "Facade-integrated": (9, 18),      # Near vertical
    "Adjusted": (27, 45)               # Custom category
}

In [13]:
def compute_trig_ranges(range_tuple):
    min_deg, max_deg = range_tuple
    min_rad, max_rad = np.radians(min_deg), np.radians(max_deg)
    sin_range = (np.sin(min_rad), np.sin(max_rad))
    cos_range = (np.cos(min_rad), np.cos(max_rad))
    return sin_range, cos_range

In [14]:
# Map categories to degree ranges and compute trig ranges
data["TiltDegreeRange"] = data["MainOrientationTiltAngle"].map(angle_ranges)
data["TiltSinRange"], data["TiltCosRange"] = zip(*data["TiltDegreeRange"].map(compute_trig_ranges))

In [15]:
data.drop(["MainOrientationTiltAngle"], axis=1, inplace=True)

In [18]:
data['TimeSinceCommissioning'] = (dt.datetime.now() - data['CommissioningDate']).dt.days

In [19]:
data.drop(["RegistrationDate", "CommissioningDate"], axis=1, inplace=True)

In [20]:
data['Efficiency'] = data['NetRatedPower'] / data['GrossPower']

In [21]:
data['PowerPerModule'] = data['GrossPower'] / data['NumberOfModules']

In [22]:
data.drop(["TiltDegreeRange"], axis=1, inplace=True)

In [23]:
data.head()

,Country,State,City,District,GrossPower,MainOrientation,NetRatedPower,FeedInType,AssignedActivePowerInverter,NumberOfModules,...,CommissioningMonth,CommissioningDay,RegistrationYear,RegistrationMonth,RegistrationDay,TiltSinRange,TiltCosRange,TimeSinceCommissioning,Efficiency,PowerPerModule
0,Germany,Nordrhein-Westfalen,Münster,Münster,3.96,South,3.96,Full Feed-in,4.0,22.0,...,7,20,2019,2,1,"(0.3090169943749474, 0.5877852522924731)","(0.9510565162951535, 0.8090169943749475)",6369,1.000000,0.180
1,Germany,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,7.41,South,7.41,Partial Feed-in,8.3,38.0,...,1,31,2019,1,31,"(0.3090169943749474, 0.5877852522924731)","(0.9510565162951535, 0.8090169943749475)",4347,1.000000,0.195
2,Germany,Brandenburg,Havelland,Nauen,5.04,South,5.00,Partial Feed-in,5.0,16.0,...,2,19,2019,1,31,"(0.3090169943749474, 0.5877852522924731)","(0.9510565162951535, 0.8090169943749475)",3233,0.992063,0.315
3,Germany,Bavaria,Regensburg,Pentling,6.36,South-West,6.00,Partial Feed-in,6.0,24.0,...,12,16,2019,1,31,"(0.3090169943749474, 0.5877852522924731)","(0.9510565162951535, 0.8090169943749475)",2932,0.943396,0.265
4,Germany,Saarland,Saarlouis,Saarlouis,7.20,West,7.20,Partial Feed-in,7.6,30.0,...,12,8,2019,1,31,"(0.3090169943749474, 0.5877852522924731)","(0.9510565162951535, 0.8090169943749475)",4767,1.000000,0.240


In [24]:
data.columns

Index(['Country', 'State', 'City', 'District', 'GrossPower', 'MainOrientation',
       'NetRatedPower', 'FeedInType', 'AssignedActivePowerInverter',
       'NumberOfModules', 'Location', 'UnitOperationalStatus',
       'UniformOrientationAndTiltAngle', 'CommissioningYear',
       'CommissioningMonth', 'CommissioningDay', 'RegistrationYear',
       'RegistrationMonth', 'RegistrationDay', 'TiltSinRange', 'TiltCosRange',
       'TimeSinceCommissioning', 'Efficiency', 'PowerPerModule'],
      dtype='object')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3986931 entries, 0 to 4642260
Data columns (total 24 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Country                         object 
 1   State                           object 
 2   City                            object 
 3   District                        object 
 4   GrossPower                      float64
 5   MainOrientation                 object 
 6   NetRatedPower                   float64
 7   FeedInType                      object 
 8   AssignedActivePowerInverter     float64
 9   NumberOfModules                 float64
 10  Location                        object 
 11  UnitOperationalStatus           object 
 12  UniformOrientationAndTiltAngle  float64
 13  CommissioningYear               int64  
 14  CommissioningMonth              int64  
 15  CommissioningDay                int64  
 16  RegistrationYear                int64  
 17  RegistrationMonth          

In [ ]:
data.to_csv('solar_visualization.csv', index=False)

In [27]:
!git add solar_dataframe_visualization.ipynb

In [28]:
!git commit -m "Formatting Dataframe for Visualization"

[master 8ba2877] Formatting Dataframe for Visualization
 1 file changed, 877 insertions(+)
 create mode 100644 solar_dataframe_visualization.ipynb


In [29]:
!git push origin master

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 4.46 KiB | 1.11 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:Gokhan-Dede/data-optional-experimenting.git
   3bf6da9..8ba2877  master -> master
